In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.api.types import CategoricalDtype
import scipy.stats as stats
import statsmodels.formula.api as sm

print(pd.__version__)

%matplotlib inline

0.22.0


In [3]:
# Read table

mercari_data = pd.read_table("./train.tsv", index_col = 'train_id', dtype = {'item_condition_id':CategoricalDtype(categories = [str(i) for i in range(1,6)], ordered = True), 'category_name':'category', 'brand_name': 'category', 'shipping':'category'})

/Users/Abe/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:466: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [28]:
cat1, cat2, cat3, a = mercari_data.category_name.str.split("/", 3).str
mercari_data['category_1'] = cat1
mercari_data['category_2'] = cat2
mercari_data['category_3'] = cat3
for col in ['category_1', 'category_2', 'category_3']:
    mercari_data[col] = mercari_data[col].astype('category')

In [43]:
mercari_data['log_price'] = np.log(mercari_data.price + 1)

In [81]:
KLdivergence_data = pd.Series()

mercari_mean = mercari_data.log_price.mean()
#Sample variance
mercari_sv = mercari_data.log_price.std()**2

for category in mercari_data.category_3.cat.categories.values:
    category_log_price_data = mercari_data[mercari_data.category_3 == category].log_price
    category_mean = category_log_price_data.mean()
    category_sv = category_log_price_data.std()**2
    # STD approximates variance for large N
    #Assuming Gaussian distribution, calculate KL-divergence based on mean and variance of fit curve
    if category_sv == 0: continue
    KLdivergence = (np.log(mercari_sv/category_sv))+(category_sv+(category_mean-mercari_mean)**2/(2*mercari_sv))-(1/2)
    KLdivergence_data[category] = KLdivergence
    
KLdivergence_data

100 Years or Older                -0.077392
50 To 75 Years                     0.015929
75 To 100 Years                    0.016729
A-Line                             0.680873
Above Knee, Mini                   0.309311
Accessories                        0.239646
Accessory                          1.432405
Aceo                                    NaN
Action Figure                      0.146049
Action Figures & Statues           0.163242
Action, Adventure                 -0.022663
Activity Centers & Entertainers    0.489590
Advertisement                      1.136938
Afghan                             0.083386
Air Conditioners                   2.324592
Air Fresheners                     0.638688
Air Purifiers                      0.424648
All Other Sports                  -0.035780
Amplifiers & Effects               0.634877
Animal                             0.208650
Animals                            4.483309
Animation                          0.321396
Apron                           

In [74]:
# Trying to generalize getting KLdivergence data into function - incomplete
data_by_category = mercari_data.groupby('category_3')
# SV is sample variance
def KLdivergence(new_data, old_data):
    new_data_mean = new_data.mean()
    old_data_mean = old_data.mean()
    new_data_sv = new_data.std()**2
    old_data_sv = old_data.std()**2
    # STD approximates variance for large N
    #Assuming Gaussian distribution, calculate KL-divergence based on mean and variance of fit curve
    KLdivergence = (np.log(mercari_sv/category_sv))+(category_sv+(category_mean-mercari_mean)**2/(2*mercari_sv))-(1/2)
data_by_category.apply()

TypeError: apply() missing 1 required positional argument: 'func'

In [88]:
# Double-checking KL-divergence values for reasonableness
category_stats = pd.DataFrame(columns = pd.Index(['mean', 'std', 'mean_diff', 'std_diff', 'n']))
for category in mercari_data.category_3.cat.categories.values:
    category_price_data = mercari_data[mercari_data.category_3 == category].log_price
    if category_price_data.count() < 100: continue
    category_stats.at[category, 'n'] = category_price_data.count()
    category_stats.at[category, 'mean'] = category_price_data.mean()
    category_stats.at[category, 'mean_diff'] = category_price_data.mean() - mercari_data.log_price.mean()
    category_stats.at[category, 'std'] = category_price_data.std()
    category_stats.at[category, 'std_diff'] = category_price_data.std() - mercari_data.log_price.std()
    
category_stats['KL_divergence'] = KLdivergence_data
print('mean: ' + str(mercari_data.log_price.mean()))
print('std: ' + str(mercari_data.log_price.std()))
category_stats

mean: 2.979059002252257
std: 0.7492093835694243


,mean,std,mean_diff,std_diff,n,KL_divergence
100 Years or Older,2.97001,1.00208,-0.00905138,0.252875,140,-0.077392
A-Line,3.09451,0.465352,0.11545,-0.283857,756,0.680873
"Above Knee, Mini",2.91433,0.599344,-0.0647295,-0.149865,20082,0.309311
Accessories,2.6704,0.680929,-0.308655,-0.0682801,10297,0.239646
Action Figure,3.06437,0.690783,0.0853074,-0.0584262,5354,0.146049
Action Figures & Statues,3.00021,0.675865,0.0211492,-0.0733445,12524,0.163242
Air Fresheners,2.70198,0.496198,-0.277084,-0.253011,441,0.638688
Animal,2.72854,0.682227,-0.25052,-0.0669823,253,0.208650
Animation,2.38039,0.807575,-0.598668,0.0583661,2954,0.321396
Area Rugs & Pads,3.25017,0.606706,0.271112,-0.142503,214,0.355513
